In [7]:
import random
import heapq
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Tuple
from scipy import stats
import matplotlib.pyplot as plt

# ==========================================
# CONFIGURATION
# ==========================================
TICKER = "RY.TO"
TICK_SIZE = 0.01
INITIAL_PRICE = 100.00
SIM_DURATION = 390 * 60  # 6.5 trading hours in seconds (23400 seconds)

# Default Stochastic Parameters (can be overridden per experiment)
DEFAULT_ARRIVAL_RATE = 0.8    # Noise orders per second
DEFAULT_CANCEL_RATE = 1/60.0  # Avg 60 seconds before cancellation
DEFAULT_MM_WAKEUP = 5.0       # Market Maker updates quotes every 5 seconds
DEFAULT_MM_SKEW = 0.05        # How aggressively MM skews price to fix inventory
DEFAULT_N_MARKET_MAKERS = 1   # Number of market makers
DEFAULT_SPREAD_WIDTH = 2      # Ticks wide for MM quotes

# ==========================================
# DATA STRUCTURES
# ==========================================

@dataclass(order=True)
class Order:
    price: float
    timestamp: float
    quantity: int
    side: str = field(compare=False)
    order_type: str = field(compare=False)
    agent_id: str = field(compare=False)
    order_id: str = field(compare=False)
    
    def __post_init__(self):
        if self.price is not None:
            self.price = round(self.price / TICK_SIZE) * TICK_SIZE

@dataclass(order=True)
class Event:
    time: float
    event_type: str
    data: Dict[str, Any] = field(default_factory=dict, compare=False)

# ==========================================
# ORDER BOOK LOGIC
# ==========================================

class OrderBook:
    def __init__(self):
        self.bids: List[Order] = [] 
        self.asks: List[Order] = []
        self.trades: List[Dict[str, Any]] = []
        
    def get_best_bid(self):
        return self.bids[0].price if self.bids else None
    
    def get_best_ask(self):
        return self.asks[0].price if self.asks else None
    
    def get_mid_price(self):
        bb = self.get_best_bid()
        ba = self.get_best_ask()
        if bb and ba:
            return (bb + ba) / 2
        return bb if bb else (ba if ba else INITIAL_PRICE)
    
    def get_spread(self):
        bb = self.get_best_bid()
        ba = self.get_best_ask()
        return (ba - bb) if (bb and ba) else 0.0
    
    def get_book_depth(self):
        """Returns total quantity on bid and ask sides"""
        bid_depth = sum(o.quantity for o in self.bids)
        ask_depth = sum(o.quantity for o in self.asks)
        return bid_depth, ask_depth

    def add_order(self, order: Order, current_time: float):
        if order.side == 'buy':
            self._match_buy(order, current_time)
        else:
            self._match_sell(order, current_time)
            
    def _match_buy(self, order: Order, current_time: float):
        while order.quantity > 0 and self.asks:
            best_ask = self.asks[0]
            if order.order_type == 'market' or order.price >= best_ask.price:
                self._execute(order, best_ask, current_time)
            else:
                break
        
        if order.quantity > 0 and order.order_type == 'limit':
            self.bids.append(order)
            self.bids.sort(key=lambda x: (-x.price, x.timestamp))
            
    def _match_sell(self, order: Order, current_time: float):
        while order.quantity > 0 and self.bids:
            best_bid = self.bids[0]
            if order.order_type == 'market' or order.price <= best_bid.price:
                self._execute(best_bid, order, current_time)
            else:
                break

        if order.quantity > 0 and order.order_type == 'limit':
            self.asks.append(order)
            self.asks.sort(key=lambda x: (x.price, x.timestamp))

    def _execute(self, bid: Order, ask: Order, current_time: float):
        qty = min(bid.quantity, ask.quantity)
        price = bid.price if bid.timestamp < ask.timestamp else ask.price
        
        self.trades.append({
            'time': current_time,
            'price': price,
            'quantity': qty,
            'buy_agent': bid.agent_id,
            'sell_agent': ask.agent_id,
            'buy_order_id': bid.order_id,
            'sell_order_id': ask.order_id
        })
        
        bid.quantity -= qty
        ask.quantity -= qty
        
        if bid.quantity == 0 and bid in self.bids:
            self.bids.remove(bid)
        if ask.quantity == 0 and ask in self.asks:
            self.asks.remove(ask)

    def cancel_order(self, order: Order):
        if order.quantity > 0:
            if order.side == 'buy' and order in self.bids:
                self.bids.remove(order)
                return True
            elif order.side == 'sell' and order in self.asks:
                self.asks.remove(order)
                return True
        return False

# ==========================================
# SIMULATION ENGINE
# ==========================================

class Simulation:
    def __init__(self, config: Dict[str, Any] = None):
        self.config = config or {}
        self.current_time = 0.0
        self.event_queue: List[Event] = [] 
        self.book = OrderBook()
        self.order_id_counter = 0
        
        # Configuration
        self.arrival_rate = self.config.get('arrival_rate', DEFAULT_ARRIVAL_RATE)
        self.cancel_rate = self.config.get('cancel_rate', DEFAULT_CANCEL_RATE)
        self.mm_wakeup = self.config.get('mm_wakeup', DEFAULT_MM_WAKEUP)
        self.mm_skew = self.config.get('mm_skew', DEFAULT_MM_SKEW)
        self.n_market_makers = self.config.get('n_market_makers', DEFAULT_N_MARKET_MAKERS)
        self.spread_width = self.config.get('spread_width', DEFAULT_SPREAD_WIDTH)
        
        # Tracking dictionaries for metrics
        self.order_tracker: Dict[str, Dict] = {}  # order_id -> {submit_time, cancel_time, fill_time, etc.}
        self.market_snapshots: List[Dict] = []  # Periodic market state
        self.mm_inventories: Dict[str, List] = {f'MM{i}': [] for i in range(self.n_market_makers)}

    def _id_generator(self) -> str:
        self.order_id_counter += 1
        return f"O_{self.order_id_counter}"

    def schedule_event(self, event: Event):
        heapq.heappush(self.event_queue, event)

    def _schedule_noise_trader_event(self, agent_index: int):
        delay = random.expovariate(self.arrival_rate)
        next_time = self.current_time + delay
        self.schedule_event(Event(
            time=next_time,
            event_type='NOISE_TRADE',
            data={'agent_index': agent_index}
        ))

    def _schedule_mm_wakeup(self, mm_id: str):
        next_time = self.current_time + self.mm_wakeup
        self.schedule_event(Event(
            time=next_time,
            event_type='MM_WAKEUP',
            data={'mm_id': mm_id}
        ))

    def _schedule_cancellation_timer(self, order: Order):
        wait_time = random.expovariate(self.cancel_rate)
        next_time = self.current_time + wait_time
        self.schedule_event(Event(
            time=next_time,
            event_type='CANCEL_ORDER',
            data={'order': order}
        ))
        
    def _schedule_market_monitor(self):
        next_time = self.current_time + 60.0
        self.schedule_event(Event(
            time=next_time,
            event_type='MONITOR_MARKET',
            data={}
        ))

    def handle_noise_trade(self, agent_index: int):
        mid = self.book.get_mid_price()
        side = 'buy' if random.random() < 0.5 else 'sell'
        oid = self._id_generator()
        
        # Track order submission
        self.order_tracker[oid] = {
            'submit_time': self.current_time,
            'side': side,
            'agent': 'Noise',
            'initial_quantity': 100,
            'filled_quantity': 0,
            'status': 'active'
        }
        
        if random.random() < 0.30:
            # Market Order
            price = 999999.0 if side == 'buy' else 0.0
            order = Order(price, self.current_time, 100, side, 'market', 'Noise', oid)
            self.book.add_order(order, self.current_time)
            
            # Check if filled immediately
            if order.quantity == 0:
                self.order_tracker[oid]['fill_time'] = self.current_time
                self.order_tracker[oid]['filled_quantity'] = 100
                self.order_tracker[oid]['status'] = 'filled'
        else:
            # Limit Order
            offset = random.randint(1, 5) * TICK_SIZE
            price = mid - offset if side == 'buy' else mid + offset
            order = Order(price, self.current_time, 100, side, 'limit', 'Noise', oid)
            self.book.add_order(order, self.current_time)
            
            # Check if filled immediately
            if order.quantity == 0:
                self.order_tracker[oid]['fill_time'] = self.current_time
                self.order_tracker[oid]['filled_quantity'] = 100
                self.order_tracker[oid]['status'] = 'filled'
            else:
                # Schedule cancellation
                self._schedule_cancellation_timer(order)
            
        self._schedule_noise_trader_event(agent_index)

    def handle_market_maker_wakeup(self, mm_id: str):
        # Cancel previous quotes from this MM
        orders_to_cancel = [o for o in self.book.bids + self.book.asks if o.agent_id == mm_id]
        for o in orders_to_cancel:
            self.book.cancel_order(o)

        # Calculate inventory
        filled_buys = sum(t['quantity'] for t in self.book.trades if t['buy_agent'] == mm_id)
        filled_sells = sum(t['quantity'] for t in self.book.trades if t['sell_agent'] == mm_id)
        inventory = filled_buys - filled_sells
        
        # Track inventory over time
        self.mm_inventories[mm_id].append({
            'time': self.current_time,
            'inventory': inventory
        })
        
        # Determine pricing with inventory skew
        mid = self.book.get_mid_price()
        skew = -(inventory * self.mm_skew * TICK_SIZE)
        
        bid_price = mid - (self.spread_width * TICK_SIZE) + skew
        ask_price = mid + (self.spread_width * TICK_SIZE) + skew
        
        # Place quotes
        oid_b = self._id_generator()
        oid_a = self._id_generator()
        
        buy_order = Order(bid_price, self.current_time, 200, 'buy', 'limit', mm_id, oid_b)
        sell_order = Order(ask_price, self.current_time, 200, 'sell', 'limit', mm_id, oid_a)
        
        self.book.add_order(buy_order, self.current_time)
        self.book.add_order(sell_order, self.current_time)
        
        self._schedule_mm_wakeup(mm_id)

    def handle_cancellation(self, order_to_cancel: Order):
        success = self.book.cancel_order(order_to_cancel)
        if success and order_to_cancel.order_id in self.order_tracker:
            tracker = self.order_tracker[order_to_cancel.order_id]
            tracker['cancel_time'] = self.current_time
            tracker['status'] = 'cancelled'
            tracker['filled_quantity'] = tracker['initial_quantity'] - order_to_cancel.quantity
        
    def handle_market_monitor(self):
        bid_depth, ask_depth = self.book.get_book_depth()
        self.market_snapshots.append({
            'time': self.current_time,
            'mid_price': self.book.get_mid_price(),
            'spread': self.book.get_spread(),
            'bid_depth': bid_depth,
            'ask_depth': ask_depth,
            'n_trades': len(self.book.trades)
        })
        self._schedule_market_monitor()

    def run(self, verbose=False):
        if verbose:
            print(f"--- Simulating {TICKER} ---")
            print(f"Config: λ={self.arrival_rate}, θ={self.cancel_rate}, MMs={self.n_market_makers}")
        
        # Initialize processes
        self._schedule_noise_trader_event(1)
        self._schedule_noise_trader_event(2)
        for i in range(self.n_market_makers):
            self._schedule_mm_wakeup(f'MM{i}')
        self._schedule_market_monitor()

        # Main event loop
        while self.event_queue and self.current_time < SIM_DURATION:
            next_event: Event = heapq.heappop(self.event_queue)
            self.current_time = next_event.time
            
            if self.current_time >= SIM_DURATION:
                break

            event_type = next_event.event_type
            
            if event_type == 'NOISE_TRADE':
                self.handle_noise_trade(next_event.data['agent_index'])
            elif event_type == 'MM_WAKEUP':
                self.handle_market_maker_wakeup(next_event.data['mm_id'])
            elif event_type == 'CANCEL_ORDER':
                self.handle_cancellation(next_event.data['order'])
            elif event_type == 'MONITOR_MARKET':
                self.handle_market_monitor()
        
        # Update any partially filled orders
        for oid, tracker in self.order_tracker.items():
            if tracker['status'] == 'active':
                # Find corresponding fills in trades
                filled = sum(t['quantity'] for t in self.book.trades 
                           if t.get('buy_order_id') == oid or t.get('sell_order_id') == oid)
                tracker['filled_quantity'] = filled
                if filled > 0:
                    tracker['status'] = 'partial'
                    
        return self.get_metrics()

    def get_metrics(self) -> Dict[str, Any]:
        """Compute all metrics for this replication"""
        
        # Order execution metrics
        waiting_times = []
        fill_rates = []
        
        for oid, tracker in self.order_tracker.items():
            if tracker.get('fill_time'):
                wait = tracker['fill_time'] - tracker['submit_time']
                waiting_times.append(wait)
            
            fill_rate = tracker['filled_quantity'] / tracker['initial_quantity']
            fill_rates.append(fill_rate)
        
        # Market metrics
        spreads = [s['spread'] for s in self.market_snapshots if s['spread'] > 0]
        mid_prices = [s['mid_price'] for s in self.market_snapshots]
        
        # MM metrics
        mm_metrics = {}
        for mm_id in [f'MM{i}' for i in range(self.n_market_makers)]:
            mm_buys = sum(t['quantity'] for t in self.book.trades if t['buy_agent'] == mm_id)
            mm_sells = sum(t['quantity'] for t in self.book.trades if t['sell_agent'] == mm_id)
            mm_inv = mm_buys - mm_sells
            
            cash = 0
            for t in self.book.trades:
                if t['buy_agent'] == mm_id:
                    cash -= t['price'] * t['quantity']
                if t['sell_agent'] == mm_id:
                    cash += t['price'] * t['quantity']
            
            final_mid = self.book.get_mid_price()
            pnl = cash + (mm_inv * final_mid)
            
            mm_metrics[mm_id] = {
                'inventory': mm_inv,
                'pnl': pnl,
                'trades': mm_buys + mm_sells
            }
        
        return {
            'total_trades': len(self.book.trades),
            'avg_waiting_time': np.mean(waiting_times) if waiting_times else 0,
            'median_waiting_time': np.median(waiting_times) if waiting_times else 0,
            'avg_fill_rate': np.mean(fill_rates),
            'avg_spread': np.mean(spreads) if spreads else 0,
            'mid_price_volatility': np.std(mid_prices) if len(mid_prices) > 1 else 0,
            'final_mid_price': self.book.get_mid_price(),
            'mm_metrics': mm_metrics,
            'raw_waiting_times': waiting_times,
            'raw_spreads': spreads,
            'raw_mid_prices': mid_prices
        }

# ==========================================
# EXPERIMENTAL FRAMEWORK
# ==========================================

def run_replications(config: Dict[str, Any], n_reps: int = 30, seeds: List[int] = None) -> pd.DataFrame:
    """Run multiple replications with different random seeds"""
    if seeds is None:
        seeds = range(n_reps)
    
    results = []
    for i, seed in enumerate(seeds[:n_reps]):
        random.seed(seed)
        np.random.seed(seed)
        
        sim = Simulation(config)
        metrics = sim.run(verbose=(i == 0))
        
        # Flatten metrics for dataframe
        row = {
            'replication': i,
            'seed': seed,
            'total_trades': metrics['total_trades'],
            'avg_waiting_time': metrics['avg_waiting_time'],
            'median_waiting_time': metrics['median_waiting_time'],
            'avg_fill_rate': metrics['avg_fill_rate'],
            'avg_spread': metrics['avg_spread'],
            'mid_price_volatility': metrics['mid_price_volatility'],
            'final_mid_price': metrics['final_mid_price']
        }
        
        # Add MM metrics
        for mm_id, mm_data in metrics['mm_metrics'].items():
            row[f'{mm_id}_pnl'] = mm_data['pnl']
            row[f'{mm_id}_inventory'] = mm_data['inventory']
            row[f'{mm_id}_trades'] = mm_data['trades']
        
        results.append(row)
    
    return pd.DataFrame(results)

def compute_confidence_intervals(df: pd.DataFrame, confidence: float = 0.95) -> pd.DataFrame:
    """Compute confidence intervals for all numeric columns"""
    summary = []
    
    for col in df.select_dtypes(include=[np.number]).columns:
        if col not in ['replication', 'seed']:
            data = df[col].dropna()
            mean = data.mean()
            sem = stats.sem(data)
            ci = stats.t.interval(confidence, len(data)-1, loc=mean, scale=sem)
            
            summary.append({
                'metric': col,
                'mean': mean,
                'std': data.std(),
                'ci_lower': ci[0],
                'ci_upper': ci[1],
                'ci_width': ci[1] - ci[0]
            })
    
    return pd.DataFrame(summary)

def run_experiment(experiment_name: str, configs: List[Dict], n_reps: int = 30):
    """Run a comparative experiment across multiple configurations"""
    print(f"\n{'='*60}")
    print(f"EXPERIMENT: {experiment_name}")
    print(f"{'='*60}\n")
    
    all_results = {}
    
    for config in configs:
        config_name = config.get('name', str(config))
        print(f"\nRunning configuration: {config_name}")
        print(f"Parameters: {config}")
        
        df = run_replications(config, n_reps)
        ci_df = compute_confidence_intervals(df)
        
        all_results[config_name] = {
            'raw_data': df,
            'summary': ci_df
        }
        
        print(f"\nResults for {config_name}:")
        print(ci_df.to_string(index=False))
    
    return all_results

# ==========================================
# MAIN: RUN EXPERIMENTS
# ==========================================

if __name__ == "__main__":
    
    # Experiment 1: Effect of Number of Market Makers
    print("\n" + "="*80)
    print("EXPERIMENT 1: Effect of Number of Market Makers")
    print("="*80)
    
    mm_configs = [
        {'name': '1_MM', 'n_market_makers': 1},
        {'name': '2_MMs', 'n_market_makers': 2},
        {'name': '3_MMs', 'n_market_makers': 3}
    ]
    
    mm_results = run_experiment("Market Maker Count", mm_configs, n_reps=30)
    
    # Experiment 2: Effect of Arrival Rate (Market Volatility)
    print("\n" + "="*80)
    print("EXPERIMENT 2: Effect of Arrival Rate")
    print("="*80)
    
    arrival_configs = [
        {'name': 'Low_λ=0.5', 'arrival_rate': 0.5},
        {'name': 'Medium_λ=0.8', 'arrival_rate': 0.8},
        {'name': 'High_λ=1.2', 'arrival_rate': 1.2}
    ]
    
    arrival_results = run_experiment("Arrival Rate", arrival_configs, n_reps=30)
    
    # Experiment 3: Effect of MM Spread Width
    print("\n" + "="*80)
    print("EXPERIMENT 3: Effect of MM Spread Width")
    print("="*80)
    
    spread_configs = [
        {'name': 'Tight_1tick', 'spread_width': 1},
        {'name': 'Medium_2ticks', 'spread_width': 2},
        {'name': 'Wide_3ticks', 'spread_width': 3}
    ]
    
    spread_results = run_experiment("Spread Width", spread_configs, n_reps=30)
    
    print("\n" + "="*80)
    print("ALL EXPERIMENTS COMPLETE")
    print("="*80)
    
    # Summary comparison
    print("\n\nKEY FINDINGS SUMMARY:")
    print("\n1. Market Maker Count Impact:")
    for name, result in mm_results.items():
        avg_spread = result['summary'][result['summary']['metric'] == 'avg_spread']['mean'].values[0]
        mm_pnl = result['summary'][result['summary']['metric'] == 'MM0_pnl']['mean'].values[0]
        print(f"   {name}: Avg Spread = ${avg_spread:.4f}, MM PnL = ${mm_pnl:.2f}")
    
    print("\n2. Arrival Rate Impact:")
    for name, result in arrival_results.items():
        volatility = result['summary'][result['summary']['metric'] == 'mid_price_volatility']['mean'].values[0]
        trades = result['summary'][result['summary']['metric'] == 'total_trades']['mean'].values[0]
        print(f"   {name}: Volatility = ${volatility:.4f}, Total Trades = {trades:.0f}")
    
    print("\n3. Spread Width Impact:")
    for name, result in spread_results.items():
        mm_pnl = result['summary'][result['summary']['metric'] == 'MM0_pnl']['mean'].values[0]
        fill_rate = result['summary'][result['summary']['metric'] == 'avg_fill_rate']['mean'].values[0]
        print(f"   {name}: MM PnL = ${mm_pnl:.2f}, Fill Rate = {fill_rate:.2%}")


EXPERIMENT 1: Effect of Number of Market Makers

EXPERIMENT: Market Maker Count


Running configuration: 1_MM
Parameters: {'name': '1_MM', 'n_market_makers': 1}
--- Simulating RY.TO ---
Config: λ=0.8, θ=0.016666666666666666, MMs=1


c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc



Results for 1_MM:
              metric          mean          std      ci_lower      ci_upper     ci_width
        total_trades  16506.933333   250.555099  16413.374522  16600.492145   187.117623
    avg_waiting_time      0.000000     0.000000           NaN           NaN          NaN
 median_waiting_time      0.000000     0.000000           NaN           NaN          NaN
       avg_fill_rate      0.706431     0.005561      0.704354      0.708507     0.004153
          avg_spread      0.039183     0.000757      0.038900      0.039466     0.000565
mid_price_volatility      0.261500     0.127331      0.213954      0.309046     0.095092
     final_mid_price    100.185500     0.630002     99.950253    100.420747     0.470493
             MM0_pnl -11085.583333   381.045727 -11227.868146 -10943.298520   284.569626
       MM0_inventory      3.333333    85.028731    -28.416917     35.083583    63.500500
          MM0_trades 655656.666667 19850.782723 648244.262578 663069.070755 14824.808176

R

c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc



Results for 2_MMs:
              metric          mean          std      ci_lower      ci_upper     ci_width
        total_trades  22100.800000   240.338468  22011.056141  22190.543859   179.487718
    avg_waiting_time      0.000000     0.000000           NaN           NaN          NaN
 median_waiting_time      0.000000     0.000000           NaN           NaN          NaN
       avg_fill_rate      0.770043     0.003899      0.768587      0.771499     0.002912
          avg_spread      0.039446     0.001020      0.039065      0.039827     0.000762
mid_price_volatility      0.487082     0.188877      0.416554      0.557610     0.141056
     final_mid_price    100.169667     1.151167     99.739814    100.599520     0.859706
             MM0_pnl  -9946.516667   372.145654 -10085.478138  -9807.555196   277.922942
       MM0_inventory     13.333333    97.320421    -23.006709     49.673376    72.680085
          MM0_trades 835180.000000 13736.370047 830050.755128 840309.244872 10258.489745
 

...

c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc



Results for High_λ=1.2:
              metric          mean          std      ci_lower      ci_upper    ci_width
        total_trades  24039.166667   193.367273  23966.962140  24111.371193  144.409053
    avg_waiting_time      0.000000     0.000000           NaN           NaN         NaN
 median_waiting_time      0.000000     0.000000           NaN           NaN         NaN
       avg_fill_rate      0.712565     0.003876      0.711118      0.714013    0.002895
          avg_spread      0.040976     0.000792      0.040680      0.041272    0.000592
mid_price_volatility      0.166035     0.052487      0.146436      0.185634    0.039198
     final_mid_price    100.098167     0.482871     99.917860    100.278474    0.360614
             MM0_pnl -14805.566667   337.418387 -14931.560763 -14679.572570  251.988193
       MM0_inventory     26.666667    82.768199     -4.239487     57.572820   61.812307
          MM0_trades 805786.666667 12174.102908 801240.781931 810332.551402 9091.769471

EXPERI

c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2241: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\abrau\anaconda3\Lib\site-packages\scipy\stats\_distn_infrastructure.py:2242: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc



Results for Tight_1tick:
              metric          mean         std      ci_lower      ci_upper    ci_width
        total_trades  18845.566667  118.368415  18801.367174  18889.766159   88.398985
    avg_waiting_time      0.000000    0.000000           NaN           NaN         NaN
 median_waiting_time      0.000000    0.000000           NaN           NaN         NaN
       avg_fill_rate      0.766495    0.003221      0.765292      0.767697    0.002406
          avg_spread      0.047597    0.001183      0.047155      0.048039    0.000884
mid_price_volatility      0.383902    0.147409      0.328858      0.438945    0.110087
     final_mid_price    100.414833    0.859814    100.093774    100.735893    0.642120
             MM0_pnl -17177.416667  245.075642 -17268.929415 -17085.903918  183.025498
       MM0_inventory    -16.666667   98.552746    -53.466867     20.133533   73.600400
          MM0_trades 898490.000000 6740.733102 895972.968893 901007.031107 5034.062213

Running configur

KeyboardInterrupt: 